# Train an IntegratedML model on ED Readmit likelihood Dataset
## Use JDBC to connect to InterSystems IRIS database
**NOTE: This Notebook will not run as-is!**
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict ED Readmits in the next 90 days
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

### 1. Get jdbc connection and cursor

In [1]:
import pandas as pd

In [2]:
%run -i '../Initializations/Conns.py'

### 2. Create and specify the source data table(s)

In [3]:
#Use this block to create a starting data set that you can/will build upon.
#NOTE: It is always useful to have a unique identifier in the data
TargetTable = 'Data.EDEncsPredB90View'
TrainTable = 'Data.EDEncsTraining'
TestTable = 'Data.EDEncsTesting'

### 3. Review the data to ensure the Target variable and Independent variables are in good standing.

In [4]:
tKeep()
import pandas as pd
from IPython.display import display

df = pd.read_sql("select top 5 * from Data.PatientCostData", iconn)
print(df)
print(df.columns)

   ID  DRID  HPS   Age Gend Mar InsRel PlanCode  T30  T60  ...  S180  S365  \
0   1     1  4.0  52.0    F   M      S      L05  0.0  0.0  ...  53.0  87.0   
1   2     2  2.0  48.0    F   M      1    13947  0.0  0.0  ...   0.0   1.0   
2   3     3  1.0  66.0    F   M      S      W01  NaN  NaN  ...   NaN   NaN   
3   4     4  NaN  62.0    M   D      S      S20  NaN  NaN  ...   NaN   NaN   
4   5     5  NaN  51.0    M   S      S      W01  0.0  1.0  ...   NaN   NaN   

   Gen  dGen  Med  dMed  Appt  dAppt   Rx  dRx  
0  9.0   7.0  9.0   3.0   0.0    0.0  2.0  2.0  
1  NaN   NaN  NaN   NaN   NaN    NaN  NaN  NaN  
2  NaN   NaN  NaN   NaN   NaN    NaN  NaN  NaN  
3  NaN   NaN  NaN   NaN   NaN    NaN  NaN  NaN  
4  NaN   NaN  NaN   NaN   NaN    NaN  NaN  NaN  

[5 rows x 75 columns]
Index(['ID', 'DRID', 'HPS', 'Age', 'Gend', 'Mar', 'InsRel', 'PlanCode', 'T30',
       'T60', 'T90', 'E30', 'E90', 'E180', 'E365', 'I180', 'I365', 'O30',
       'O90', 'O180', 'O365', 'ObsHt', 'dObsHt', 'ObsWt', 'dO

### Drop and unwanted fields

In [5]:
df = df.drop(['T30', 'T60', 'T90'], axis = 1)
Usable = str(list(df.columns)).replace("', '", ",")[2:-2]
Usable

'ID,DRID,HPS,Age,Gend,Mar,InsRel,PlanCode,E30,E90,E180,E365,I180,I365,O30,O90,O180,O365,ObsHt,dObsHt,ObsWt,dObsWt,ObsBMI,dObsBMI,ObsBSA,dObsBSA,ObsTemp,dObsTemp,ObsPulse,dObsPulse,ObsBPS,dObsBPS,ObsBPD,dObsBPD,ObsO2,dObsO2,Labs,dLabs,Meds,dMeds,Rads,dRads,Vax,dVax,HypChol_E78,dHypChol_E78,TII_E11,dTII_E11,GenEnc_Z00,dGenEnc_Z00,Scrn_Z13,dScrn_Z13,Couns_Z71,dCouns_Z71,OWt_E66,dOWt_E66,HypThy_E03,dHypThy_E03,Scrn_Z12,dScrn_Z12,S30,S90,S180,S365,Gen,dGen,Med,dMed,Appt,dAppt,Rx,dRx'

In [6]:
icurs.execute(' \
    create or replace view %s as \
        select case when t90 > 0 then 1 else 0 end as B90, %s \
        from Data.PatientCostData' % (TargetTable, Usable))
df1 = pd.read_sql('SELECT COUNT(*) as Recs FROM %s' % TargetTable, iconn)
TargetVar = 'B90'
display(df1)

,Recs
0,126633


In [7]:
Distro = pd.read_sql('select %s, count(*) as Recs from %s group by %s' % (TargetVar, TargetTable, TargetVar), iconn)
Distro

,B90,Recs
0,0,120181
1,1,6452


### 4. Assess the probability of your target and sample accordingly into split training and testing datasets

In [8]:
#we want to split the data into Training (80%) and Test (20%), ...
# but also reduce the ratio of Negative (ED Enc = 0) to Positive
Train = 0.8
TVRatio = 2
PT_List = pd.read_sql('select DRID, %s from %s order by %s, DRID' % (TargetVar, TargetTable, TargetVar), iconn)
PT_List.index = PT_List['DRID']

In [9]:
#Create the lists, check the ratios, and create the "In lists":
TrainList = PT_List[PT_List[TargetVar] == 0].sample(int(Distro['Recs'].loc[1]*TVRatio*Train)) \
    .append(PT_List[PT_List[TargetVar] == 1].sample(int(Distro['Recs'].loc[1]*Train)))
TrainList['Flag'] = 1
TrainList.index = TrainList['DRID']
print(TrainList.pivot_table(index = TargetVar, values = 'DRID', aggfunc = 'count'))
#NOTE: It is IMPERATIVE that Test does NOT contain any Train data
TestList = PT_List.join(TrainList['Flag'], how = 'left')
TestList = TestList[(TestList['Flag'] != 1)]
TestList = TestList[(TestList[TargetVar] == 1)].append(TestList[TestList[TargetVar] == 0].sample(int(len(TestList)*0.25)))
print(TestList.pivot_table(index = TargetVar, values = 'DRID', aggfunc = 'count'))
TrainIns = str(list(TrainList['DRID']))[1:-1]
TestIns = str(list(TestList['DRID']))[1:-1]

      DRID
B90       
0    10323
1     5161
      DRID
B90       
0    27787
1     1291


In [10]:
# Training set view
icurs.execute(''' \
    CREATE or replace VIEW %s AS \
        SELECT * FROM %s 
        WHERE DRID in (%s)''' \
    % (TrainTable, TargetTable, TrainIns))
# Prediction set
icurs.execute(''' \
    CREATE or replace VIEW %s AS \
        SELECT * FROM %s 
        WHERE DRID in (%s)''' \
    % (TestTable, TargetTable, TestIns))

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [11]:
try:
    icurs.execute("CREATE MODEL NewEncModel PREDICTING (%s) FROM %s" % (TargetVar, TrainTable))
except:
    icurs.execute("DROP MODEL NewEncModel")
    icurs.execute("CREATE MODEL NewEncModel PREDICTING (%s) FROM %s" % (TargetVar, TrainTable))

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [15]:
icurs.execute("set ml configuration %AutoML")
icurs.execute("TRAIN MODEL NewEncModel as NewEncModel_Auto")
icurs.execute("set ml configuration DRCfg")
icurs.execute("TRAIN MODEL NewEncModel as NewEncModel_DR")

DatabaseError: java.sql.SQLException: [SQLCODE: <-400>:<Fatal error occurred>]
[Location: <ServerLoop>]
[%msg: <ERROR #2800: %ML General Error 'Training failed:': error){"message": "Deployment limit reached"}.>]

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [16]:
pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", iconn)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,NoShowModel,NoShowModel_t2,AutoML,2020-10-26 15:57:18.731000,classification,"ModelType:Random Forest, Package:sklearn, Prob..."
1,NewEncModelTD,NewEncModelAutoML,AutoML,2020-10-27 01:46:39.459000,classification,"ModelType:Logistic Regression, Package:sklearn..."


### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [14]:
TestSet = pd.read_sql("SELECT PREDICT(NewEncModel use NewEncModel_Auto) AS PredictedEncs, \
                  case when B90 = 1 then 1 end AS ActualPos, \
                  case when B90 != 1 then 0 end AS ActualNeg \
                  FROM %s" % (TestTable), iconn)

DatabaseError: Execution failed on sql 'SELECT PREDICT(NewEncModel use NewEncModel_Auto) AS PredictedEncs,                   case when B90 = 1 then 1 end AS ActualPos,                   case when B90 != 1 then 0 end AS ActualNeg                   FROM Data.EDEncsTesting': java.sql.SQLException: [SQLCODE: <-181>:<Model or Trained Model not found>]
[Location: <Prepare>]
[%msg: < Model 'NewEncModel_Auto' not found>]

In [ ]:
print(pd.pivot_table(TestSet, index = 'PredictedEncs', values = ['ActualPos', 'ActualNeg'], aggfunc = 'count'))
print('Accuracy: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == TestSet['ActualPos']) \
            | (TestSet['PredictedEncs'] == TestSet['ActualNeg'])])/len(TestSet))*100))+'%')
print('Misclassification Rate: '+str(round((len(TestSet[(TestSet['PredictedEncs'] != TestSet['ActualPos']) \
            & (TestSet['PredictedEncs'] != TestSet['ActualNeg'])])/len(TestSet))*100))+'%')
print('%FP: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == 1) & (TestSet['ActualNeg'] == 0)])/ \
            len(TestSet[TestSet['ActualNeg'] == 0]))*100))+'%')
print('%FN: '+str(round((len(TestSet[(TestSet['PredictedEncs'] == 0) & (TestSet['ActualPos'] == 1)])/ \
            len(TestSet[TestSet['ActualPos'] == 1]))*100))+'%')

### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [ ]:
#icurs.execute("VALIDATE model NewEncModel FROM Data.EDEncsTesting")
#df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", iconn)
#df5
#df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
#display(df6)